In [1]:
import pandas as pd
import numpy as np
import re
import datetime

In [107]:
data = pd.read_csv(
    'assets/tiktokMediasFranco.csv'
)

data

,compte,dateHeure,description,nbCommentaires,nbLikes,nbVues,nbPartages,texteSurimposé,url,durée,musiqueTitre,musiqueArtiste
0,infobref,2022-11-26 10:06:51,Les prix de l’#immobilier au Canada vont baiss...,0,17,551,6,NaN,https://www.tiktok.com/@infobref/video/7170341...,166,son original,InfoBref
1,infobref,2022-11-18 10:04:06,#TWITTER survivra-t-il à #ELONMUSK? À retenir ...,0,1,137,0,NaN,https://www.tiktok.com/@infobref/video/7167370...,595,son original - InfoBref,InfoBref
2,infobref,2022-11-11 13:42:35,Le chemin du métavers est pénible pour Meta | ...,0,1,25,0,NaN,https://www.tiktok.com/@infobref/video/7164830...,577,son original,InfoBref
3,infobref,2022-11-05 08:25:36,#ElonMusk fait le ménage chez #Twitter | Nouve...,0,1,88,0,NaN,https://www.tiktok.com/@infobref/video/7162507...,534,son original,InfoBref
4,infobref,2022-10-30 14:28:43,À retenir dans l'actualité des #affaires: on v...,0,0,11,0,NaN,https://www.tiktok.com/@infobref/video/7160374...,587,son original,InfoBref
...,...,...,...,...,...,...,...,...,...,...,...,...
13238,hugodecrypte,2019-11-22 11:52:26,70 chars de guerre défilent dans les rues en B...,8,933,18400,4,Chaque année en Belgique. Il y a le + grand ra...,https://www.tiktok.com/@hugodecrypte/video/676...,29,son original,hugodecrypte
13239,hugodecrypte,2019-11-21 08:40:41,Emma Watson en 8 dates #foryou #pourtoi #emmaw...,5,773,5144,5,EMMA WATSON \nen 8 dates. 2001 : son premier f...,https://www.tiktok.com/@hugodecrypte/video/676...,28,On Top Of The World,Imagine Dragons
13240,hugodecrypte,2019-11-20 05:58:04,Ils lisent 500 pages en 1h30 !Reportage sur no...,32,1153,11000,32,Ils lisent 500 pages \nen 1h30. C’est le champ...,https://www.tiktok.com/@hugodecrypte/video/676...,31,son original,hugodecrypte
13241,hugodecrypte,2019-11-18 04:04:04,Omar Sy en 8 dates #foryou #pourtoi #hugodecry...,6,617,8763,3,OMAR SY \nen 8 dates. 1995 : sa rencontre \nav...,https://www.tiktok.com/@hugodecrypte/video/676...,28,Believer,Imagine Dragons


# Preprocess global

In [113]:
# Transformation du dataset pour l'ajout d'une colonne pays
def add_countries(data):
    media_to_country = {
        'infobref':"Canada",
        'latribune':"Canada",
        'lavoixdunord':"France",
        'ledevoir':"Canada",
        'heidi.news':"Suisse",
        'tdg.ch':"Suisse",
        'letemps':"Suisse",
        'leprogres.lyon':"France",
        'sudouestfr':"France",
        'journalmetro':"Canada",
        'radio.canada.info':"Canada",
        'rtlinfo':"Belgique",
        'majmonactu':"Canada",
        'noovo.info':"Canada",
        'ouestfrance':"France",
        'radpointca':"Canada",
        '24heuresca':"Canada",
        '_urbania':"Canada",
        'tvasports':"Canada",
        'rds.ca':"Canada",
        'lefigarofr':"France",
        'miseajour':"Belgique",
        'rtsinfo':"Suisse",
        'loopsider':"France",
        'lequipe':"France",
        'tf1info':"France",
        'leparisien':"France",
        'franceinfo':"France",
        'lemondefr':"France",
        'rmcsport':"France",
        'bfmtv':"France",
        'konbini':"France",
        'brutofficiel':"France",
        'hugodecrypte':"France"
    }

    data['pays'] = [media_to_country[media] for media in data['compte']]

In [206]:
# Transformation du dataset pour la séparation de la date et heure
def seperate_datetime(data):
    data["dateHeure"] = pd.to_datetime(data["dateHeure"], format="%Y-%m-%d %H:%M:%S")
    data['date'] = [dateheure.date() for dateheure in data['dateHeure']]
    data['date_str'] = [dateheure.strftime('%Y-%m') for dateheure in data['dateHeure']]
    data['time'] = [dateheure.time() for dateheure in data['dateHeure']]
    data['hour'] = [dateheure.strftime('%H') for dateheure in data['dateHeure']]
    data.drop(columns=['dateHeure'], inplace=True)

In [207]:
# Transformation du dataset pour extraire les tags des descriptions
def extract_tags_from_descriptions(data):
    
    def desc_to_tags(desc):
        if type(desc) != str :
            return []
        desc = desc.replace('#', ' #').split()
        tags = [re.sub(r'[^\w\s]', '', tag.lower()) for tag in desc if tag.startswith("#")]
        return tags

    data['tags'] = data['description'].apply(desc_to_tags)


In [208]:
# Transformation du dataset pour regrouper les durées en catégories distrinctes
def group_length(data):
    data['durée'] = ['<' + str(np.ceil(d / 60) * 60) if d > 0 else '<60' for d in data['durée'] ]

In [209]:
# Transformation du dataset pour enlever les colonnes inutiles
def remove_unused_columns(data):
    data.drop(columns=['texteSurimposé', 'url', 'musiqueTitre', 'musiqueArtiste', 'description'], inplace = True)

In [222]:
data = pd.read_csv(
    'assets/tiktokMediasFranco.csv'
)

add_countries(data)
seperate_datetime(data)
extract_tags_from_descriptions(data)
group_length(data)
remove_unused_columns(data)

data

,compte,nbCommentaires,nbLikes,nbVues,nbPartages,durée,pays,date,date_str,time,hour,tags
0,infobref,0,17,551,6,<180.0,Canada,2022-11-26,2022-11,10:06:51,10,"[immobilier, immobilierqc, immobilierquebec, i..."
1,infobref,0,1,137,0,<600.0,Canada,2022-11-18,2022-11,10:04:06,10,"[twitter, elonmusk, actualité, affaires, finan..."
2,infobref,0,1,25,0,<600.0,Canada,2022-11-11,2022-11,13:42:35,13,"[nouvelles, affaires, finances, meta]"
3,infobref,0,1,88,0,<540.0,Canada,2022-11-05,2022-11,08:25:36,08,"[elonmusk, twitter, actualités, affaires, affa..."
4,infobref,0,0,11,0,<600.0,Canada,2022-10-30,2022-10,14:28:43,14,[affaires]
...,...,...,...,...,...,...,...,...,...,...,...,...
13238,hugodecrypte,8,933,18400,4,<60.0,France,2019-11-22,2019-11,11:52:26,11,"[pourtoi, foryou]"
13239,hugodecrypte,5,773,5144,5,<60.0,France,2019-11-21,2019-11,08:40:41,08,"[foryou, pourtoi, emmawatson, harrypotter]"
13240,hugodecrypte,32,1153,11000,32,<60.0,France,2019-11-20,2019-11,05:58:04,05,"[foryou, pourtoi, hugodecrypte]"
13241,hugodecrypte,6,617,8763,3,<60.0,France,2019-11-18,2019-11,04:04:04,04,"[foryou, pourtoi, hugodecrypte, tiktok, omarsy]"


# Diagramme à bulle

In [223]:
def filter_by_year(data, year):
    assert year in [2019,2020,2021,2022], "Incorrect year"
    return data[(data['date'] >= datetime.date(year,1,1)) & (data['date'] <= datetime.date(year,12,31))]

In [224]:
# Transformation des données pour le graphique à bulle en fonction des tags
def explode_tags(data):
    # Exploser les listes de tags en lignes
    data_tag = data.explode('tags')
    
    # Regrouper les données par tags
    data_tag = pd.DataFrame(data_tag.groupby([data_tag['tags']], as_index=False).agg({'nbVues':'mean', 'nbPartages':'mean','nbCommentaires':'mean','compte':'size'}).round())
    data_tag.rename(columns={"compte": "n_post"}, inplace = True)
    
    # Sort la liste en fonction des tags les plus courant
    data_tag = data_tag.sort_values('n_post', ascending = False)
    
    # Retourner seulement le top 100 tags
    return data_tag[:100]

In [225]:
# Transformation des données pour le graphique à bulle
# group_by_column peut avoir 3 valeurs différentes : "compte", "durée" ou "tags"
# year peur avoir 5 valeurs différentes : 2019, 2020, 2021, 2022 ou "all"
def bubble_graph(data, group_by_column, year = "all"):
    if year != 'all':
        data = filter_by_year(data, year)
        
    if group_by_column == 'compte':
        return pd.DataFrame(data.groupby([data[group_by_column], data['pays']], as_index=False).mean().round())
    
    if group_by_column == 'durée':
        return pd.DataFrame(data.groupby([data[group_by_column]], as_index=False).mean().round())
    
    if group_by_column == 'tags':
        return explode_tags(data)

In [227]:
bubble_graph(data, 'compte', 2020)

,compte,pays,nbCommentaires,nbLikes,nbVues,nbPartages
0,franceinfo,France,238.0,20383.0,401486.0,265.0
1,heidi.news,Suisse,0.0,6.0,195.0,4.0
2,hugodecrypte,France,503.0,42105.0,260984.0,642.0
3,konbini,France,131.0,15171.0,111100.0,104.0
4,lavoixdunord,France,10.0,100.0,3384.0,2.0
5,ledevoir,Canada,0.0,13.0,266.0,3.0
6,lefigarofr,France,3.0,239.0,16940.0,3.0
7,lemondefr,France,178.0,21234.0,267104.0,241.0
8,lequipe,France,37.0,4362.0,69264.0,32.0
9,letemps,Suisse,1.0,45.0,1004.0,3.0


# Diagramme à ligne

In [228]:
# Transformation du dataset pour filtrer les données en fonction des options sélectionnées
# selection peut être None s'il n'y a pas de filter à faire, où un tuple (colonne, valeur) pour faire un filtre sur les données
def filter_graph(data, selected):
    assert len(selected) == 2, 'Format de la selection incorrect'
    assert selected[0] in data.columns, 'selected[0] (' + str(selected[0]) + ') ne fait pas partie des colonnes'
    
    if selected[0] == 'tags' :
        data = data.explode('tags')
        
    return data[data[selected[0]] == selected[1]]

In [229]:
# Transformation du dataset pour le graphique à ligne
# selection peut être None s'il n'y a pas de filter à faire, où un tuple (colonne, valeur) pour faire un filtre sur les données
# year peur avoir 5 valeurs différentes : 2019, 2020, 2021, 2022 ou "all"
def line_graph(data, selected=None, year = "all"):
    if year != 'all':
        data = filter_by_year(data, year)
        
    if selected is not None:
        data = filter_graph(data, selected)
    
    # Group by date 
    data = pd.DataFrame(data.groupby([data['date_str']], as_index=False).agg({'compte':'size'}))
    data.rename(columns={"compte": "n_post"}, inplace = True)
    
    return data

In [230]:
line_graph(data, ['tags', 'news'], 2021)

,date_str,n_post
0,2021-01,1
1,2021-02,4
2,2021-03,3
3,2021-04,2
4,2021-05,2
5,2021-06,6
6,2021-07,1
7,2021-08,4
8,2021-09,7
9,2021-10,8


# Pie Chart en forme d'horloge 24h, yikes

In [231]:
def pie_chart(data):
    data = pd.DataFrame(data.groupby([data['hour']], as_index=False).agg({'compte':'size'}))
    data.rename(columns={"compte": "n_post"}, inplace = True)
    
    return data

In [233]:
pie_chart(data)

,hour,n_post
0,00,23
1,01,52
2,02,244
3,03,282
4,04,568
5,05,722
6,06,790
7,07,574
8,08,620
9,09,843
